In [ ]:
import os
os.chdir('..')
print os.getcwd()

In [ ]:
from keras.models import load_model
from keras.preprocessing import image as image_utils
import numpy as np
import matplotlib.pyplot as plt
import main.utils.config as conf

In [ ]:
os.chdir('test/')

In [ ]:
# loading the models for testing
path = '../main/'
test_imgs_path = 'samples/'
test_pain_imgs_path = 'pain_samples/'
emotions_model = load_model(path+'model_emo_InceptionV3_nogcloud_44ep.h5')
pain_model_adam = load_model(path+'model_pain_nogcloud_adam_120ep.h5')
pain_model_rmsprop = load_model(path+'model_pain_nogcloud_rmsprop_240ep.h5')
pain_model_datagen = load_model(path+'model_pain_nogcloud_datagen_240ep.h5')
sr_model = load_model(path+'model_finalSR_3densemodel_CrossVal.h5')

In [ ]:
## PNG/JPG parse and pain detection and measurement

for f in os.listdir(test_imgs_path):
    fileName = os.path.join(test_imgs_path, f)
    
    # img to matrix
    image = image_utils.load_img(fileName, target_size=(conf.PIC_DIM_PAIN, conf.PIC_DIM_PAIN))
    image = image_utils.img_to_array(image).astype(np.float32)
    image = image/ 255.
    print "colored image ", fileName, " :"
    plt.imshow(image)
    plt.show()
    
    # img to gray scale matrix
    gray_image = image_utils.load_img(fileName, target_size=(conf.PICTURE_DIM_EMOTION, conf.PICTURE_DIM_EMOTION))
    gray_image = image_utils.img_to_array(gray_image).astype(np.float32)
    gray_image = gray_image/ 255.
    gray_image = np.dot(gray_image[..., :3], [0.299, 0.587, 0.114])
    print "gray image ", fileName, " :"
    plt.imshow(gray_image)
    plt.show()
    gray_image = np.reshape(gray_image, (conf.PICTURE_DIM_EMOTION, conf.PICTURE_DIM_EMOTION, 1))

    # emotion prediction
    emotion_prediction = emotions_model.predict(np.array([gray_image]), batch_size = 1)[0]
    
    print 
    print emotion_prediction
    print
    print "predicted emotion:", conf.emotion.items()[np.argmax(emotion_prediction)][1], "with prob:", np.max(emotion_prediction)
    
    # If pain emotion predicted, predict its level
    if np.argmax(emotion_prediction) == 7 :
        
        painLevel_prediction = pain_model_adam.predict(np.array([image]), batch_size = 1)[0]
        
        print "predicted pain level:", np.argmax(painLevel_prediction)

In [ ]:
for f in os.listdir(test_pain_imgs_path):
    fileName = os.path.join(test_pain_imgs_path, f)
    
    # img to matrix
    image = image_utils.load_img(fileName, target_size=(conf.PIC_DIM_PAIN, conf.PIC_DIM_PAIN))
    image = image_utils.img_to_array(image).astype(np.float32)
    image = image/ 255.
    print "colored image ", fileName, " :"
    plt.imshow(image)
    plt.show()
    
    painLevel_prediction = pain_model_rmsprop.predict(np.array([image]), batch_size = 1)[0]
    
    print 
    print painLevel_prediction
    print
    print "predicted pain level:", np.argmax(painLevel_prediction), "with prob:", np.max(painLevel_prediction)
    